In [1]:
import numpy as np
import cv2        

In [2]:
def image_stats(image):
    #compute the mean and standard deviation of each channel
    (l, a, b) = cv2.split(image)
    (lMean, lStd) = (l.mean(), l.std())
    (aMean, aStd) = (a.mean(), a.std())
    (bMean, bStd) = (b.mean(), b.std())
    
    #return the color statistics
    return (lMean, lStd, aMean, aStd, bMean, bStd)

In [3]:

def color_transfer(source, target):
        #convert the images from the RGB to L*a*b* color space, being
        # sure to utilizing the floating point data type (note: OpenCV
        # expects floats to be 32-bit, so use that instead of 64-bit)
        source = cv2.cvtColor(source, cv2.COLOR_BGR2LAB).astype("float32")
        target = cv2.cvtColor(target, cv2.COLOR_BGR2LAB).astype("float32")
        
        #compute color statistics for the source and target images
        (lMeanSrc, lStdSrc, aMeanSrc, aStdSrc, bMeanSrc, bStdSrc) = image_stats(source)
        (lMeanTar, lStdTar, aMeanTar, aStdTar, bMeanTar, bStdTar) = image_stats(target)
        
        #subsctract the means from the target image
        (l,a,b) = cv2.split(target)
        l -= lMeanTar
        a -= aMeanTar
        b -= bMeanTar
        
        #scale bu the standard deviations
        l = (lStdTar / lStdSrc) * l
        a = (aStdTar / aStdSrc) * a
        b = (bStdTar / bStdSrc) * b
        
        #clip the pixel intensities to [0, 255] if the fall outside
        #this range
        l = np.clip(l, 0, 255)
        a = np.clip(a, 0, 255)
        b = np.clip(b, 0, 255)
        
        #merge the channels together and convert back to the RGB color
        #space, being sure to utilize the 8-bit unsigned integer data
        # type
        transfer = cv2.merge([l, a, b])
        transfer = cv2.cvtColor(transfer.astype("uint8"), cv2.COLOR_LAB2BGR)
        
        #return the color tranferred image
        return transfer

In [ ]:
def show_image(title, image, width = 300):
    # resize the image to have a constant width, just to
    # make displaying the images take up less screen real
    # estate
    r = width / float(image.shape[1])
    dim = (width, int(image.shape[0] * r))
    resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)

    # show the resized image
    cv2.imshow(title, resized)

In [ ]:
imgSrc = cv2.imread("C:/Users/Yaroslav/Pictures/For projects/images.jpg")
imgTar = cv2.imread("C:/Users/Yaroslav/Pictures/For projects/creepy_woods3.jpg")


transfer = color_transfer(imgSrc, imgTar)

show_image("Source", imgSrc)
show_image("Target", imgTar)
show_image("Transfer", transfer)
cv2.waitKey(0)